# 防跌小車

使用紅外線模組確認地面是否存在，如果不存在，就退後或是轉彎

----


In [14]:
%websocketconnect --password 1234 ws://192.168.50.224:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


### 測試紅外線模組

- 0:有偵測到
- 1:無偵測到

In [4]:
#%sendtofile main.py

import socket
from machine import Pin
import time

D6 = 12
Pir = Pin(D6, Pin.IN)

for i in range(20):
    print('['+str(Pir.value())+']', end='')
    time.sleep(1)

....

***OSError [遠端主機已強制關閉一個現存的連線。]



## 防跌小車

In [11]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = Pin(D4, Pin.OUT)
for i in range(5):
    led.value(not led.value())
    time.sleep(0.2)
    
# ir
Pir = Pin(D6, Pin.IN)

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1
REV = 0

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(0)     #r
    #motor_a2.value(FWD)
    motor_b1.value(1)     #l
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(0)
    #motor_b2.value(REV)

# 快速右轉
def car_right2():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)    

# 快速左轉
def car_left2():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


# go~
car_fwd()

last_distance = 0
while(True):

    print(Pir.value())
    if Pir.value()==0:#有地面
        car_fwd()
    else:
        car_rev()
        time.sleep(1)
        car_right2()
        time.sleep(0.5)

    time.sleep(0.1)

Sent 95 lines (1645 bytes) to main.py.


## 練習題：防跌、避障自走車

請試著把目前學到的超音波元件、紅外線元件整合在一起，完成你的智慧自走車


- TODO: 三向避障 + 紅外線防掉落

In [15]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())

# ir
Pir = Pin(D6, Pin.IN)    

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1
REV = 0

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(0)     #r
    #motor_a2.value(FWD)
    motor_b1.value(1)     #l
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(0)
    #motor_b2.value(REV)

# 快速右轉
def car_right2():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)    

# 快速左轉
def car_left2():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58


len_f = None
distance_r = None
distance_l = None

# go~
car_fwd()
while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>25:
        car_fwd()

        print(Pir.value())
        if Pir.value()==0:#有地面
            car_fwd()
        else:
            car_rev()
            time.sleep(1)
            car_right2()
            time.sleep(0.5)

    else:
        car_stop()
        time.sleep(0.5)
        car_right2()
        time.sleep(0.25)
        car_stop()
        time.sleep(0.5)
        distance_r = ping(trigPin=D7,echoPin=D8)
        print("distance_r = " + str(distance_r))
        time.sleep(0.1)

        car_left2()
        time.sleep(0.5)
        car_stop()
        time.sleep(0.5)
        distance_l = ping(trigPin=D7,echoPin=D8)
        print("distance_l = " + str(distance_l))
        time.sleep(0.1)
        
        car_right2()
        time.sleep(0.25)
        car_stop()
        time.sleep(0.5)
        distance = ping(trigPin=D7,echoPin=D8)
        print("distance = " + str(distance))
        time.sleep(0.5)
        
        if(distance_r>distance_l):
            car_right2()
        else:
            car_left2()
        time.sleep(0.25)

    time.sleep(0.1)

Sent 169 lines (3581 bytes) to main.py.
